In [1]:
# NEW SPHEROS
import sphero
import time

In [2]:
# Widgets
# from __future__ import print_function

# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets

In [163]:
# Initialize Sphero manager with addresses of local spheros
manager = sphero.SpheroManager()
manager._name_cache = {"68:86:E7:05:0C:4B": "Sphero-YYP",
                       "68:86:E7:05:19:AD": "Sphero-RYR",
                       "68:86:E7:05:12:26": "Sphero-RPB"
                      
                      
                      }

NBOTS = 4   # Number of spheros 



In [164]:
# use this code snippet when working with new spheros that you have not encountered before.
def on_new_sphero(device, NBOTS=NBOTS):
    """
        NBOTS = number of robots in the flock
        Note that this presently refers to a global "manager" object. This method should be moved inside the native
        library later.
    """
    print "Found " + device.bt_name
    
    # Terminate search when all expected bots are found
    if len(manager._spheros.keys()) == NBOTS:
        print "Found all {} spheros".format(NBOTS)
        manager.stop_auto_search()
        
        
        
# Callback
manager.set_sphero_found_cb(on_new_sphero)

# Construct list of devices in system bluetooth collection
# Your bluetooth network cannot contain anything with name of "Sphero-" prefix
devices = []

manager.start_auto_search()

Starts auto search


In [160]:
# Verify that these are the robots we are looking for
for name, device in manager._spheros.iteritems():
    print "{}: {}".format(name, device.bt_addr)
    devices.append(device)
    

In [166]:
for m in manager._spheros.iteritems():
    print m


('Sphero-RPB', <sphero.core.SpheroAPI object at 0x0000000004C9EE10>)
('Sphero-YYP', <sphero.core.SpheroAPI object at 0x0000000004AE9358>)
('Sphero-RYR', <sphero.core.SpheroAPI object at 0x0000000004C9E4E0>)


In [177]:
bot2 = [b for b in manager._spheros.iteritems()][1][1]



In [7]:
bot = [b for b in manager._spheros.iteritems()][2]

In [8]:
bot = bot[1]

In [178]:
bot2.connect()



try 0
An exception of type IOError occurred. Arguments:
('A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n',)
try 1
An exception of type IOError occurred. Arguments:
('A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n',)
try 2
Connected!


True

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 415, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 299, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Pytho

In [9]:
bot.connect()

try 0
Connected!


True

In [130]:
bot.set_rgb(0, 255,0 )


In [91]:
import time
import math

# PID controller for sphero

class pidController():

    def __init__(self):
        self.isRunning = False
        self.epsilon   = 0.1 # some small bound? 
    def reset(self):
        self.isRunning = False

    # later may find that P controller is sufficient
    # Documentation is here: 
    
    # headingCMD = desiredHeading + (headingKp*currErrorHeading)
    # velocityCMD = velocityKp*distance
    # http://ctms.engin.umich.edu/CTMS/index.php?example=Introduction&section=ControlPID
    def getPIDSpeed(self, distance, speed, Kp, Ki, Kd, stopRadius=3, 
                   maxSpeed=70, minSpeed=30, resumeSpeed=50):
        
        
        
        """
            Other variables self explanatory?
            Kp: Proportional controller gain
            Ki: Intergral controller gain
            Kd: Derivative controller gain
            stopRadius: If robot is within this distance of object, try to stop.
            distance: Distance in centimeters between robot and desired point
            resumeSpeed: If robot comes to standstill, this overcomes inertia
        """

        if not self.isRunning:
            self.isRunning = True
            self.prevU = 0
            self.prev2E = 0
            self.prevE = 0
            self.prev2T = time.time()
            self.prevT = time.time()


        currentT = time.time()
        deltaT   = currentT - self.prevT
        deltaT2  = self.prevT - self.prev2T

        # u is the speed that you return
        # breaking behavior may be different from just turning motor off
        # maybe may need matlab to tune the controller constants
        
        if distance < stopRadius:
            u = 0
        else:  # Robot too far away, must keep moving!

            # PID equation
            # Make select mode using PID switch
            if (deltaT < self.epsilon) or (deltaT2 < self.epsilon):
                u = self.prevU + Kp * (distance - self.prevE) + Ki*deltaT*distance
            else:
                u = self.prevU + Kp * (distance - self.prevE) + Ki*deltaT*distance + Kd * \
                                         (((distance - self.prevE) / deltaT) - \
                                         ((self.prevE - self.prev2E) / deltaT2))
                            
            # If robot has stopped moving, reset it
            if (speed < 2) and (u < resumeSpeed):
                u = resumeSpeed

        # Update internal vars
        self.prevU = u

        self.prev2E = self.prevE
        self.prevE = distance
        self.prev2T = self.prevT
        self.prevT = currentT

        print "candidate u: {}".format(u)
        # Handle saturation
        if (u > maxSpeed):
            return maxSpeed
        elif (u < minSpeed):
            
            return minSpeed
        else:
            return u

In [57]:
resp = bot.roll(90, 180)

In [129]:
def normalizeAngle(angle):
    if angle < 0:
        return 360 + angle
    
    else:
        
        return angle
    
Kp = 0.25
Ki = 0.08
Kd = 0.08


# replace with calls to the locator
response = bot.read_locator()  # refactor with call to webcam
currentX = response.x_pos
currentY = response.y_pos

currentSpeed = response.sog  # lookup this param

controller = pidController()
stopRadius = 3


# Basic closed loop controller
startTime = time.time()

# run for 30 seconds
while ((time.time() - startTime ) < 2):
    targetX = 0
    targetY = 0
    
    
    # Angle to distance
    deltaX = targetX - currentX
    deltaY = targetY - currentY
    angle = normalizeAngle(math.degrees(math.atan2(deltaX, deltaY)))
    distance = math.sqrt(deltaX * deltaX + deltaY * deltaY)
    
    while distance > stopRadius and (((time.time() - startTime ) < 20)):
        outSpeed = controller.getPIDSpeed(distance, currentSpeed, Kp, Ki, Kd)
        
        # roll the sphero, make use of the request object
        print("Dist {} outSpeed {} at {} degrees: {},{}".format(distance, outSpeed, angle, response.x_pos, response.y_pos))
        
        bot.roll(outSpeed, angle)

        # get current speed
        # this can probably be refactored each time 
        time.sleep(.3)
        response = bot.read_locator()  # refactor with call to webcam
        currentX = response.x_pos
        currentY = response.y_pos
        currentSpeed = response.sog  # lookup this param
        
        
        # Repeat waypointing calculation
        deltaX = targetX - currentX
        deltaY = targetY - currentY
        
        angle = normalizeAngle(math.degrees(math.atan2(deltaX, deltaY)))
        distance = math.sqrt(deltaX * deltaX + deltaY * deltaY)
         
print response        
print "Final Distance {} ({},{})".format(distance, currentX, currentY)


# at very end, stop the robot!!
bot.roll(0,0)

candidate u 50
Dist 26.2488094968 outSpeed 50 at 319.635463427 degrees: 17,-20
candidate u 50.3227346324
Dist 24.7588368063 outSpeed 50.3227346324 at 316.636577042 degrees: 17,-18
candidate u 48.9088386884
Dist 19.7989898732 outSpeed 48.9088386884 at 315.0 degrees: 14,-14
candidate u 45.4396889254
Dist 9.21954445729 outSpeed 45.4396889254 at 310.601294645 degrees: 7,-6
candidate u 45.1056663041
Dist 3.60555127546 outSpeed 45.1056663041 at 123.690067526 degrees: -3,2
candidate u 52.569345738
Dist 15.5563491861 outSpeed 52.569345738 at 135.0 degrees: -11,11
candidate u 49.1031094079
Dist 13.416407865 outSpeed 49.1031094079 at 153.434948823 degrees: -6,12
candidate u 45.0681090687
Dist 3.60555127546 outSpeed 45.0681090687 at 146.309932474 degrees: -2,3
candidate u 50.2361719162
Dist 9.43398113206 outSpeed 50.2361719162 at 327.994616792 degrees: 5,-8
candidate u 54.8754908987
Dist 20.5182845287 outSpeed 54.8754908987 at 316.974934011 degrees: 14,-15
candidate u 51.3786842175
Dist 17.492855

In [131]:
bot.set_heading(180)



In [132]:
bot.configure_locator(0,0)




In [157]:
bot.roll(90, 250)


In [124]:
print bot.read_locator()

 xpos: 11 
 ypos: 0 
 xvel: 0 cm/sec
 yvel: 0 cm/sec
 sog: 0 cm/sec



In [175]:
# figure out which way is backwards
bot.set_back_led_output(0xff)



In [176]:
print bot.get_power_state()




 RecVer: v.1 
 PowerState: Battery low 
 Voltage: 6.6 
 NumCharges 4 
 Last charge: 3764 sec



In [170]:
bot2.set_heading(90)


In [76]:
bot.configure_locator(0,0)






In [172]:
bot2.roll(40,0)






Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 415, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 299, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1



SpheroRequestError: No response received from device before timeout

In [ ]:
bot2.disconnect()




In [ ]:
bot2.connect()

In [ ]:
bot.configure_locator(0,0)

In [ ]:
resp = bot.read_locator()
print(resp)



In [ ]:
bot.roll(speed=20, heading=270)



In [ ]:
resp = bot.read_locator()
print(resp)



In [ ]:
# Use a widget to control the direction of the sphero
def setHeadingSlider(x):
    
    return x

interact(setHeadingSlider, x=widgets.IntSlider(min=0,max=359,step=1,value=0, continuous_update=False))

In [ ]:
# dir(devices[0].get_option_flags())





In [ ]:
devices[0].connect()



In [ ]:
for d in devices:
#     d.set_rgb(0, 255,128 )
#     print d.get_rgb( )
#     d.set_back_led_output(0xFF)
    d.set_stabilization(True) # when stabliized, won't roll
    

    print d.read_locator()    
#     print d.get_power_state()
    

In [ ]:
SPEED = 0x22

for i in range(2):
    for d in devices:
        d.roll(speed=SPEED, heading=0)
                
    time.sleep(2)

    

    for d in devices:
        d.roll(speed=SPEED, heading=90)
    
    time.sleep(2)
    
    for d in devices:
        d.roll(speed=SPEED, heading=180)
    
        
    time.sleep(2)
    for d in devices:
        d.roll(speed=SPEED, heading=270)
        
    time.sleep(2)
    

Observations

- May need windows binaries to use bluetooth properly: http://www.lfd.uci.edu/~gohlke/pythonlibs/#pybluez
- 2 spheros is fine, haven't gotten 3 spheros to work yet (a different one fails each time)

Python Lib Route
- Consider trying Kulka too, more recently updated: https://github.com/zdanowiczkonrad/PythonSphero/tree/master/Tutorials
- These are the available methods: https://github.com/orbotix/DeveloperResources/blob/master/docs/Sphero_API_1.50.pdf
- Also look at python ROS driver: https://github.com/mmwise/sphero_ros/commits/groovy-devel/sphero_driver/src/sphero_driver/sphero_driver.py